In [ ]:
import PyPDF2
import os
import pdfplumber
import pandas as pd
import re

In [ ]:
listaPDF=[]
for x in os.listdir():
    if x.endswith(".pdf"):
        listaPDF.append(x)

In [ ]:
def zwracaNumerStronyzTabela(pdf):
    odczytstrony=0
    for i in range(1,len(pdf.pages)):
        indeks = len(pdf.pages)-1-i
        p0 = pdf.pages[indeks]
        text = p0.extract_text()
        if text==None:
            continue
        elif  (text.find('R0110')!=-1 and text.find('C0020')!=-1
               and text.find('R0160')==-1 and text.find('R0010')==-1 and text.find('C0030')!=-1):
            odczytstrony=indeks
        else:
            continue
    return odczytstrony

In [ ]:
def zwracaTabele(pdf,odczytStrony):
    p0 = pdf.pages[odczytStrony]
    parametry = {
                    "vertical_strategy": "lines",
                    "horizontal_strategy": "lines",
                    "explicit_vertical_lines": p0.curves, 
                    "explicit_horizontal_lines": p0.curves, 
                    "snap_tolerance": 5,
                    "join_tolerance": 3,
                    "edge_min_length": 3,
                    "min_words_vertical": 3,
                    "min_words_horizontal": 1,
                    "keep_blank_chars": True,
                    "text_tolerance": 3,
                    "text_x_tolerance": 1.5,
                    "text_y_tolerance": 1.5,
                    "intersection_tolerance": 3,
                    "intersection_x_tolerance": None,
                    "intersection_y_tolerance": None,
                }
    tabela = p0.extract_tables(parametry)
    return tabela

In [ ]:
def usuwaNone(lista):
        for a in list(lista):
            if a==None:
                lista.remove(a)
        return lista

In [ ]:
def wypisujeObserwacje(table,wyciagana_dana):
    lista=[]
    for i in range(0,len(table)):
        for j in range(0,len(table[i])):
            for k in range(0,len(table[i][j])):
                if table[i][j][k]==wyciagana_dana:
                    lista = lista + table[i][j][k+1:len(table[i][j])] 
                else:
                    continue
    return lista

In [ ]:
def sprawdzaWystepowanieWartosci(table,wartosc):
    for i in range(0,len(table)):
        for j in range(0,len(table[i])):
            for k in range(0,len(table[i][j])):
                if table[i][j][k]==wartosc:
                    return True 
                    break      
                else:
                    continue
    return False

In [ ]:
def sprawdzaWystepowanieDanychSkladki(table):
    for i in range(0,len(table)):
        for j in range(0,len(table[i])):
            for k in range(0,len(table[i][j])):
                if bool(re.search('R[0-9]', str(table[i][j][k]).replace(" ","")))==True:
                    return True 
                    break      
                else:
                    continue
    return False

In [ ]:
def zwracaIndeksy(table,wyciaganaDana):
    lista=[]
    for i in range(0,len(table)):
        for j in range(0,len(table[i])):
            for k in range(0,len(table[i][j])):
                if table[i][j][k]==wyciaganaDana:
                    lista.append(i)
                    lista.append(j)
                else:
                    continue
    return lista

In [ ]:
daneKoncowe=[]
for sprawozdanie in listaPDF:
    print(f"Analizuje sprawozdanie: {sprawozdanie}")
    print("----------------------------------")
    pdf = pdfplumber.open(sprawozdanie)
    strona = zwracaNumerStronyzTabela(pdf)
    tabela = zwracaTabele(pdf,strona)
    wyciaganeDanezTabeli = ['R0110','R0120','R0130','R0140','R0200','R0210','R0220','R0230','R0240','R0300','R0310','R0320','R0330',
                     'R0340','R0400','R0410','R0420','R0430','R0440','R0500','R0550','R1200','R1300']
    for pobieranaDana in wyciaganeDanezTabeli:
        lista = []
        lista.append(sprawozdanie)
        lista.append(pobieranaDana)
        analizowana_strona=strona
        obserwacje=[]
        obserwacje += list(usuwaNone(wypisujeObserwacje(tabela,pobieranaDana)))
        if sprawdzaWystepowanieWartosci(tabela,'Ogółem')==False:
            while (len(obserwacje)<17):
                analizowanaStrona=analizowanaStrona+1
                obserwacje2 = []
                tabela2 = zwracaTabele(pdf,analizowana_strona)
                if sprawdzaWystepowanieDanychSkladki(tabela2)==True:
                    obserwacje2 = usuwaNone(wypisujeObserwacje(tabela2,pobieranaDana))
                    obserwacje +=obserwacje2
                else:
                    indeks = zwracaIndeksy(tabela,wyciagana_dana)
                    obserwacje+=tabela2[indeks[0]][indeks[1]]

        lista+=usuwaNone(obserwacje)
        daneKoncowe.append(lista)
    